# Gradio 之日！

今天，我们将使用极其简单的 Gradio 框架来构建用户界面。

准备好迎接喜悦吧！

请注意：您的 Gradio 屏幕可能会以“深色模式”或“浅色模式”显示，具体取决于您的计算机设置。

In [ ]:
# 导入库

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [ ]:
import gradio as gr # 哦耶！

In [ ]:
# 加载名为 .env 文件中的环境变量
# 打印密钥前缀以帮助调试

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API 密钥存在，前缀为 {openai_api_key[:8]}")
else:
    print("未设置 OpenAI API 密钥")
    
if anthropic_api_key:
    print(f"Anthropic API 密钥存在，前缀为 {anthropic_api_key[:7]}")
else:
    print("未设置 Anthropic API 密钥")

if google_api_key:
    print(f"Google API 密钥存在，前缀为 {google_api_key[:8]}")
else:
    print("未设置 Google API 密钥")

In [ ]:
# 连接到 OpenAI、Anthropic 和 Google；如果您不使用它们，请注释掉 Claude 或 Google 的行

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [ ]:
# 一个通用的系统消息 - 不再有尖刻的对抗性 AI！

system_message = "你是一个乐于助人的助手"

In [ ]:
# 让我们将对 GPT-4o-mini 的调用封装在一个简单的函数中

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content  

In [ ]:
# 这可以揭示“训练数据截止日期”，即训练数据中的最新日期

message_gpt("今天的日期是几号？")

## 用户界面时间！

In [ ]:
# 这是一个简单的函数

def shout(text):
    print(f"Shout 函数已用输入 {text} 调用")
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
# Gradio 的简洁性。这可能会以“浅色模式”显示 - 我稍后会向您展示如何使其变为深色模式。

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# 添加 share=True 意味着它可以被公开访问
# HuggingFace 的 Spaces 平台提供了更持久的托管服务，我们下周会提到
# 注意：某些杀毒软件和公司防火墙可能不允许您使用 share=True。如果您在工作或在工作网络上，我建议跳过此测试。

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# 添加 inbrowser=True 会自动打开一个新的浏览器窗口

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## 强制深色模式

Gradio 会根据浏览器和计算机的设置以浅色或深色模式显示。有一种方法可以强制 Gradio 以深色模式显示，但 Gradio 不建议这样做，因为它应该是一个用户偏好（特别是出于可访问性的原因）。但如果您希望强制您的屏幕使用深色模式，下面是方法。

In [ ]:
# 定义此变量，然后在创建 Interface 时传递 js=force_dark_mode

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
# 输入和输出

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="您的消息：", lines=6)],
    outputs=[gr.Textbox(label="回应：", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# 现在 - 将函数从“shout”更改为“message_gpt”

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="您的消息：", lines=6)],
    outputs=[gr.Textbox(label="回应：", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# 让我们使用 Markdown
# 您是否想知道，当下面的代码没有引用 system_message 时，设置它有什么区别？
# 我正在利用 system_message 是一个全局变量的特性，它在 message_gpt 函数中被使用（可以回去看看）
# 这不是一个好的软件工程实践，但在 Jupyter Lab 研发期间相当普遍！

system_message = "你是一个用 markdown 格式回应的乐于助人的助手"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="您的消息：")],
    outputs=[gr.Markdown(label="回应：")],
    flagging_mode="never"
)
view.launch()

In [ ]:
# 让我们创建一个流式返回结果的调用

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="您的消息：")],
    outputs=[gr.Markdown(label="回应：")],
    flagging_mode="never"
)
view.launch()

In [ ]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="您的消息：")],
    outputs=[gr.Markdown(label="回应：")],
    flagging_mode="never"
)
view.launch()

## 小改进

我对这段代码做了一个小改进。

之前，它有这些行：

```
for chunk in result:
  yield chunk
```

实际上，有一种更优雅的方式来实现这一点（Python 程序员可能会称之为更 'Pythonic' 的方式）：

`yield from result`


In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("未知模型")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="您的消息："), gr.Dropdown(["GPT", "Claude"], label="选择模型", value="GPT")],
    outputs=[gr.Markdown(label="回应：")],
    flagging_mode="never"
)
view.launch()

# 构建公司宣传册生成器

现在你知道怎么做了 - 很简单！

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">在阅读接下来几个单元格之前</h2>
            <span style="color:#900;">
                尝试自己动手 - 回到 week1, day5 的公司宣传册项目，并在末尾添加一个 Gradio UI。然后再来看解决方案。
            </span>
        </td>
    </tr>
</table>

In [ ]:
# 一个表示网页的类

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "未找到标题"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"网页标题：\n{self.title}\n网页内容：\n{self.text}\n\n"

In [ ]:

system_message = "你是一个助手，负责分析公司网站着陆页的内容，\
并为潜在客户、投资者和应聘者创建一份关于公司的简短宣传册。请以 markdown 格式回应。"

In [ ]:
def stream_brochure(company_name, url, model):
    prompt = f"请为 {company_name} 生成一份公司宣传册。这是他们的着陆页：\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("未知模型")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="公司名称："),
        gr.Textbox(label="着陆页 URL（包括 http:// 或 https://）"),
        gr.Dropdown(["GPT", "Claude"], label="选择模型")],
    outputs=[gr.Markdown(label="宣传册：")],
    flagging_mode="never"
)
view.launch()